In [2]:
import pandas as pd
import numpy as np

In [4]:
path = '/content/drive/MyDrive/datasets/ecommerce_returns_synthetic_data.csv'
# read (treat common text 'NaN' as missing)
df = pd.read_csv(path, na_values=["NaN", "nan", ""])
orig_columns = df.columns.tolist()            # keep list to verify later
df_original_copy = df.copy()                  # pristine backup
df = df.copy()                                # work on df

In [5]:
num_cols = ["Product_Price", "Order_Quantity", "Days_to_Return", "Discount_Applied", "User_Age"]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

In [6]:
for c in ["Order_Date", "Return_Date"]:
    if c in df.columns:
        df[c + "_dt"] = pd.to_datetime(df[c], errors="coerce")   # new column with _dt suffix

# Order-level features
df["Order_Year"] = df["Order_Date_dt"].dt.year
df["Order_Month"] = df["Order_Date_dt"].dt.month
df["Order_Weekday"] = df["Order_Date_dt"].dt.weekday   # 0=Mon
df["Order_Day"] = df["Order_Date_dt"].dt.day


In [8]:
# Flag: 1 == returned, 0 == not returned
df["Return_Flag"] = np.where(df["Return_Status"].fillna("").astype(str).str.lower().str.contains("returned"), 1, 0)

# Normalize return reason: keep original, add filled version
df["Return_Reason_filled"] = df["Return_Reason"].fillna("No Return")
df["Days_to_Return_filled"] = df["Days_to_Return"].fillna(0).astype(int)
df["Return_Date_present"] = (~df["Return_Date_dt"].isna()).astype(int)  # redundant with Return_Flag but handy


In [10]:
# Total order value (product price * quantity)
df["Total_Order_Value"] = df["Product_Price"] * df["Order_Quantity"]

# Fill other numeric NaNs with sensible defaults but keep originals
df["Product_Price_filled"] = df["Product_Price"].fillna(df["Product_Price"].median())
df["Order_Quantity_filled"] = df["Order_Quantity"].fillna(1).astype(int)
df["Discount_Applied_filled"] = df["Discount_Applied"].fillna(0.0)


In [14]:
# ---------- 0) Backup original flag (don't lose original column) ----------
if 'Return_Flag' in df.columns:
    df['Return_Flag_orig'] = df['Return_Flag']

# ---------- 1) Ensure date columns exist as datetimes (safe to run) ----------
if 'Order_Date_dt' not in df.columns and 'Order_Date' in df.columns:
    df['Order_Date_dt'] = pd.to_datetime(df['Order_Date'], errors='coerce')
if 'Return_Date_dt' not in df.columns and 'Return_Date' in df.columns:
    df['Return_Date_dt'] = pd.to_datetime(df['Return_Date'], errors='coerce')

# ---------- 2) Recompute days-to-return from dates (useful for fallback) ----------
df['Days_to_Return_calc'] = (df['Return_Date_dt'] - df['Order_Date_dt']).dt.days

# ---------- 3) Normalize the status text for reliable checks ----------
status = df['Return_Status'].fillna('').astype(str).str.strip().str.lower()

# ---------- 4) Build masks (vectorized, readable rules) ----------
# mask: status explicitly says "returned" (word boundary) but NOT "not"
mask_word_return = status.str.contains(r'\breturned\b', regex=True, na=False)
mask_word_not = status.str.contains(r'\bnot\b', regex=True, na=False)

# mask: there is a return date or a positive days-to-return (strong evidence of a real return)
mask_by_date_or_days = (
    df['Return_Date_dt'].notna()
    | (df['Days_to_Return'].notna() & (df['Days_to_Return'] > 0))
    | (df['Days_to_Return_calc'].notna() & (df['Days_to_Return_calc'] > 0))
)

# ---------- 5) Final, robust rule for corrected flag ----------
# Rule: returned if (status contains 'returned' and does not contain 'not') OR date/days indicate a return
df['Return_Flag_fixed'] = ((mask_word_return & ~mask_word_not) | mask_by_date_or_days).astype(int)

# ---------- 6) Diagnostics: what was wrong and what changed ----------
orig_flag_col = 'Return_Flag_orig' if 'Return_Flag_orig' in df.columns else 'Return_Flag'
orig_count = int(df[orig_flag_col].sum()) if orig_flag_col in df.columns else None
fixed_count = int(df['Return_Flag_fixed'].sum())

print(f"Original flagged returns (sum of {orig_flag_col}):", orig_count)
print("Fixed flagged returns (sum of Return_Flag_fixed):", fixed_count)

# Rows that were incorrectly flagged before because text contains 'not'
bad_original = df[(df[orig_flag_col]==1) & (status.str.contains(r'\bnot\b', regex=True))]
print("Originally flagged as returned but status text contains 'not':", len(bad_original))

# Rows where original flag differs from fixed
disagree = df[df['Return_Flag_fixed'] != df[orig_flag_col]]
print("Rows where original flag != fixed flag:", len(disagree))

# Sample problematic rows for quick audit (first 10)
if len(bad_original):
    print("\nSample rows originally flagged but say 'not':")
    display(bad_original.head(10))

if len(disagree):
    print("\nSample rows where original != fixed:")
    display(disagree.head(10))

# ---------- 7) Optional: replace your working 'Return_Flag' with fixed one (do only if you want) ----------
# df['Return_Flag'] = df['Return_Flag_fixed']

# ---------- 8) Optional: recompute/repair related fields (keeps originals)
# sensible Days_to_Return_filled2: prefer existing Days_to_Return, else calculated days, else 0
df['Days_to_Return_filled2'] = np.where(
    df['Days_to_Return'].notna(),
    df['Days_to_Return'],
    df['Days_to_Return_calc'].fillna(0)
).astype('int64')

# keep a clean indicator of Return_Date presence too
df['Return_Date_present2'] = df['Return_Date_dt'].notna().astype(int)


Original flagged returns (sum of Return_Flag_orig): 10000
Fixed flagged returns (sum of Return_Flag_fixed): 5052
Originally flagged as returned but status text contains 'not': 4948
Rows where original flag != fixed flag: 4948

Sample rows originally flagged but say 'not':


,Order_ID,Product_ID,User_ID,Order_Date,Return_Date,Product_Category,Product_Price,Order_Quantity,Return_Reason,Return_Status,...,Return_Reason_filled,Days_to_Return_filled,Return_Date_present,Total_Order_Value,Product_Price_filled,Order_Quantity_filled,Discount_Applied_filled,Return_Flag_orig,Days_to_Return_calc,Return_Flag_fixed
2,ORD00000002,PROD00000002,USER00000002,2023-05-06,NaN,Toys,390.03,5,NaN,Not Returned,...,No Return,0,0,1950.15,390.03,5,26.64,1,NaN,0
3,ORD00000003,PROD00000003,USER00000003,2024-08-29,NaN,Toys,401.09,3,NaN,Not Returned,...,No Return,0,0,1203.27,401.09,3,15.37,1,NaN,0
4,ORD00000004,PROD00000004,USER00000004,2023-01-16,NaN,Books,110.09,4,NaN,Not Returned,...,No Return,0,0,440.36,110.09,4,16.37,1,NaN,0
10,ORD00000010,PROD00000010,USER00000010,2024-05-26,NaN,Toys,119.00,2,NaN,Not Returned,...,No Return,0,0,238.00,119.00,2,8.72,1,NaN,0
11,ORD00000011,PROD00000011,USER00000011,2024-02-08,NaN,Home,480.48,4,NaN,Not Returned,...,No Return,0,0,1921.92,480.48,4,0.27,1,NaN,0
12,ORD00000012,PROD00000012,USER00000012,2024-06-07,NaN,Toys,216.41,3,NaN,Not Returned,...,No Return,0,0,649.23,216.41,3,30.26,1,NaN,0
13,ORD00000013,PROD00000013,USER00000013,2023-07-22,NaN,Electronics,198.78,2,NaN,Not Returned,...,No Return,0,0,397.56,198.78,2,36.58,1,NaN,0
14,ORD00000014,PROD00000014,USER00000014,2024-09-01,NaN,Electronics,146.37,4,NaN,Not Returned,...,No Return,0,0,585.48,146.37,4,28.23,1,NaN,0
16,ORD00000016,PROD00000016,USER00000016,2024-05-09,NaN,Clothing,311.08,1,NaN,Not Returned,...,No Return,0,0,311.08,311.08,1,6.66,1,NaN,0
20,ORD00000020,PROD00000020,USER00000020,2023-07-03,NaN,Clothing,6.14,1,NaN,Not Returned,...,No Return,0,0,6.14,6.14,1,47.47,1,NaN,0



Sample rows where original != fixed:


,Order_ID,Product_ID,User_ID,Order_Date,Return_Date,Product_Category,Product_Price,Order_Quantity,Return_Reason,Return_Status,...,Return_Reason_filled,Days_to_Return_filled,Return_Date_present,Total_Order_Value,Product_Price_filled,Order_Quantity_filled,Discount_Applied_filled,Return_Flag_orig,Days_to_Return_calc,Return_Flag_fixed
2,ORD00000002,PROD00000002,USER00000002,2023-05-06,NaN,Toys,390.03,5,NaN,Not Returned,...,No Return,0,0,1950.15,390.03,5,26.64,1,NaN,0
3,ORD00000003,PROD00000003,USER00000003,2024-08-29,NaN,Toys,401.09,3,NaN,Not Returned,...,No Return,0,0,1203.27,401.09,3,15.37,1,NaN,0
4,ORD00000004,PROD00000004,USER00000004,2023-01-16,NaN,Books,110.09,4,NaN,Not Returned,...,No Return,0,0,440.36,110.09,4,16.37,1,NaN,0
10,ORD00000010,PROD00000010,USER00000010,2024-05-26,NaN,Toys,119.00,2,NaN,Not Returned,...,No Return,0,0,238.00,119.00,2,8.72,1,NaN,0
11,ORD00000011,PROD00000011,USER00000011,2024-02-08,NaN,Home,480.48,4,NaN,Not Returned,...,No Return,0,0,1921.92,480.48,4,0.27,1,NaN,0
12,ORD00000012,PROD00000012,USER00000012,2024-06-07,NaN,Toys,216.41,3,NaN,Not Returned,...,No Return,0,0,649.23,216.41,3,30.26,1,NaN,0
13,ORD00000013,PROD00000013,USER00000013,2023-07-22,NaN,Electronics,198.78,2,NaN,Not Returned,...,No Return,0,0,397.56,198.78,2,36.58,1,NaN,0
14,ORD00000014,PROD00000014,USER00000014,2024-09-01,NaN,Electronics,146.37,4,NaN,Not Returned,...,No Return,0,0,585.48,146.37,4,28.23,1,NaN,0
16,ORD00000016,PROD00000016,USER00000016,2024-05-09,NaN,Clothing,311.08,1,NaN,Not Returned,...,No Return,0,0,311.08,311.08,1,6.66,1,NaN,0
20,ORD00000020,PROD00000020,USER00000020,2023-07-03,NaN,Clothing,6.14,1,NaN,Not Returned,...,No Return,0,0,6.14,6.14,1,47.47,1,NaN,0


In [15]:
df

,Order_ID,Product_ID,User_ID,Order_Date,Return_Date,Product_Category,Product_Price,Order_Quantity,Return_Reason,Return_Status,...,Return_Date_present,Total_Order_Value,Product_Price_filled,Order_Quantity_filled,Discount_Applied_filled,Return_Flag_orig,Days_to_Return_calc,Return_Flag_fixed,Days_to_Return_filled2,Return_Date_present2
0,ORD00000000,PROD00000000,USER00000000,2023-08-05,2024-08-26,Clothing,411.59,3,Changed mind,Returned,...,1,1234.77,411.59,3,45.27,1,387.0,1,387,1
1,ORD00000001,PROD00000001,USER00000001,2023-10-09,2023-11-09,Books,288.88,3,Wrong item,Returned,...,1,866.64,288.88,3,47.79,1,31.0,1,31,1
2,ORD00000002,PROD00000002,USER00000002,2023-05-06,NaN,Toys,390.03,5,NaN,Not Returned,...,0,1950.15,390.03,5,26.64,1,NaN,0,0,0
3,ORD00000003,PROD00000003,USER00000003,2024-08-29,NaN,Toys,401.09,3,NaN,Not Returned,...,0,1203.27,401.09,3,15.37,1,NaN,0,0,0
4,ORD00000004,PROD00000004,USER00000004,2023-01-16,NaN,Books,110.09,4,NaN,Not Returned,...,0,440.36,110.09,4,16.37,1,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,ORD00009995,PROD00009995,USER00009995,2023-10-20,NaN,Home,142.50,4,NaN,Not Returned,...,0,570.00,142.50,4,34.27,1,NaN,0,0,0
9996,ORD00009996,PROD00009996,USER00009996,2023-02-25,NaN,Electronics,484.63,3,NaN,Not Returned,...,0,1453.89,484.63,3,25.44,1,NaN,0,0,0
9997,ORD00009997,PROD00009997,USER00009997,2024-05-10,NaN,Toys,386.57,5,NaN,Not Returned,...,0,1932.85,386.57,5,12.67,1,NaN,0,0,0
9998,ORD00009998,PROD00009998,USER00009998,2024-02-13,NaN,Toys,129.22,1,NaN,Not Returned,...,0,129.22,129.22,1,49.97,1,NaN,0,0,0


In [16]:
# --- Define columns to keep ---
original_cols = [
    "Order_ID","Product_ID","User_ID",
    "Order_Date","Return_Date",
    "Product_Category","Product_Price","Order_Quantity",
    "Return_Reason","Return_Status",
    "User_Age","User_Gender","User_Location",
    "Payment_Method","Shipping_Method","Discount_Applied"
]

cleaned_cols = [
    "Return_Flag_fixed",
    "Days_to_Return_filled2",
    "Return_Date_present2",
    "Product_Price_filled",
    "Order_Quantity_filled",
    "Discount_Applied_filled",
    "Total_Order_Value"
]

# final feature set (no redundant columns)
keep_cols = original_cols + cleaned_cols

df_clean = df[keep_cols].copy()

print("Original dataframe shape:", df.shape)
print("Cleaned dataframe shape:", df_clean.shape)


Original dataframe shape: (10000, 36)
Cleaned dataframe shape: (10000, 23)


In [18]:
df_clean

,Order_ID,Product_ID,User_ID,Order_Date,Return_Date,Product_Category,Product_Price,Order_Quantity,Return_Reason,Return_Status,...,Payment_Method,Shipping_Method,Discount_Applied,Return_Flag_fixed,Days_to_Return_filled2,Return_Date_present2,Product_Price_filled,Order_Quantity_filled,Discount_Applied_filled,Total_Order_Value
0,ORD00000000,PROD00000000,USER00000000,2023-08-05,2024-08-26,Clothing,411.59,3,Changed mind,Returned,...,Debit Card,Next-Day,45.27,1,387,1,411.59,3,45.27,1234.77
1,ORD00000001,PROD00000001,USER00000001,2023-10-09,2023-11-09,Books,288.88,3,Wrong item,Returned,...,Credit Card,Express,47.79,1,31,1,288.88,3,47.79,866.64
2,ORD00000002,PROD00000002,USER00000002,2023-05-06,NaN,Toys,390.03,5,NaN,Not Returned,...,Debit Card,Next-Day,26.64,0,0,0,390.03,5,26.64,1950.15
3,ORD00000003,PROD00000003,USER00000003,2024-08-29,NaN,Toys,401.09,3,NaN,Not Returned,...,PayPal,Next-Day,15.37,0,0,0,401.09,3,15.37,1203.27
4,ORD00000004,PROD00000004,USER00000004,2023-01-16,NaN,Books,110.09,4,NaN,Not Returned,...,Gift Card,Standard,16.37,0,0,0,110.09,4,16.37,440.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,ORD00009995,PROD00009995,USER00009995,2023-10-20,NaN,Home,142.50,4,NaN,Not Returned,...,PayPal,Standard,34.27,0,0,0,142.50,4,34.27,570.00
9996,ORD00009996,PROD00009996,USER00009996,2023-02-25,NaN,Electronics,484.63,3,NaN,Not Returned,...,Debit Card,Express,25.44,0,0,0,484.63,3,25.44,1453.89
9997,ORD00009997,PROD00009997,USER00009997,2024-05-10,NaN,Toys,386.57,5,NaN,Not Returned,...,Credit Card,Next-Day,12.67,0,0,0,386.57,5,12.67,1932.85
9998,ORD00009998,PROD00009998,USER00009998,2024-02-13,NaN,Toys,129.22,1,NaN,Not Returned,...,Gift Card,Express,49.97,0,0,0,129.22,1,49.97,129.22


In [19]:
df = df_clean.copy()

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Order_ID                 10000 non-null  object 
 1   Product_ID               10000 non-null  object 
 2   User_ID                  10000 non-null  object 
 3   Order_Date               10000 non-null  object 
 4   Return_Date              5052 non-null   object 
 5   Product_Category         10000 non-null  object 
 6   Product_Price            10000 non-null  float64
 7   Order_Quantity           10000 non-null  int64  
 8   Return_Reason            5052 non-null   object 
 9   Return_Status            10000 non-null  object 
 10  User_Age                 10000 non-null  int64  
 11  User_Gender              10000 non-null  object 
 12  User_Location            10000 non-null  object 
 13  Payment_Method           10000 non-null  object 
 14  Shipping_Method        

In [20]:
df["Order_Date"] = pd.to_datetime(df["Order_Date"])
df["Return_Date"] = pd.to_datetime(df["Return_Date"])

In [21]:
df["Return_Reason"] = df["Return_Reason"].fillna("Not Applicable")


In [22]:
df = df.drop(columns=["Product_Price_filled", "Order_Quantity_filled", "Discount_Applied_filled"])


In [23]:
check = (df["Total_Order_Value"].round(2) == (df["Product_Price"] * df["Order_Quantity"]).round(2)).mean()
print("Match %:", check * 100)


Match %: 100.0


In [24]:
df["Days_to_Return_filled2"] = df["Days_to_Return_filled2"].apply(lambda x: abs(x))

In [25]:
cat_cols = ["Product_Category", "Return_Reason", "Return_Status",
            "User_Gender", "User_Location", "Payment_Method", "Shipping_Method"]

for col in cat_cols:
    df[col] = df[col].astype("category")


In [26]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
num_cols = ["Product_Price", "Order_Quantity", "Discount_Applied", "Total_Order_Value", "User_Age", "Days_to_Return_filled2"]

df[num_cols] = scaler.fit_transform(df[num_cols])


In [27]:
df

,Order_ID,Product_ID,User_ID,Order_Date,Return_Date,Product_Category,Product_Price,Order_Quantity,Return_Reason,Return_Status,User_Age,User_Gender,User_Location,Payment_Method,Shipping_Method,Discount_Applied,Return_Flag_fixed,Days_to_Return_filled2,Return_Date_present2,Total_Order_Value
0,ORD00000000,PROD00000000,USER00000000,2023-08-05,2024-08-26,Clothing,1.114392,-0.004336,Changed mind,Returned,0.901627,Male,City54,Debit Card,Next-Day,1.411842,1,1.531264,1,0.789174
1,ORD00000001,PROD00000001,USER00000001,2023-10-09,2023-11-09,Books,0.255540,-0.004336,Wrong item,Returned,1.554742,Female,City85,Credit Card,Express,1.587297,1,-0.532817,1,0.173866
2,ORD00000002,PROD00000002,USER00000002,2023-05-06,NaT,Toys,0.963493,1.417410,Not Applicable,Not Returned,-1.449591,Female,City30,Debit Card,Next-Day,0.114731,0,-0.712554,0,1.984889
3,ORD00000003,PROD00000003,USER00000003,2024-08-29,NaT,Toys,1.040902,-0.004336,Not Applicable,Not Returned,-0.273982,Male,City95,PayPal,Next-Day,-0.669942,0,-0.712554,0,0.736523
4,ORD00000004,PROD00000004,USER00000004,2023-01-16,NaT,Books,-0.995819,0.706537,Not Applicable,Not Returned,-0.665852,Female,City80,Gift Card,Standard,-0.600317,0,-0.712554,0,-0.538636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,ORD00009995,PROD00009995,USER00009995,2023-10-20,NaT,Home,-0.768980,0.706537,Not Applicable,Not Returned,-0.469917,Male,City40,PayPal,Standard,0.645969,0,-0.712554,0,-0.321950
9996,ORD00009996,PROD00009996,USER00009996,2023-02-25,NaT,Electronics,1.625602,-0.004336,Not Applicable,Not Returned,1.620054,Male,City62,Debit Card,Express,0.031181,0,-0.712554,0,1.155420
9997,ORD00009997,PROD00009997,USER00009997,2024-05-10,NaT,Toys,0.939276,1.417410,Not Applicable,Not Returned,0.117887,Male,City74,Credit Card,Next-Day,-0.857929,0,-0.712554,0,1.955973
9998,ORD00009998,PROD00009998,USER00009998,2024-02-13,NaT,Toys,-0.861927,-1.426083,Not Applicable,Not Returned,-0.665852,Female,City34,Gift Card,Express,1.739079,0,-0.712554,0,-1.058688


In [28]:
df = df.drop(columns=["Order_ID", "Product_ID", "User_ID"])

In [29]:
# Make sure it's datetime
df["Order_Date"] = pd.to_datetime(df["Order_Date"])
df["Return_Date"] = pd.to_datetime(df["Return_Date"])

# Extract order features
df["Order_Year"] = df["Order_Date"].dt.year
df["Order_Month"] = df["Order_Date"].dt.month
df["Order_Weekday"] = df["Order_Date"].dt.weekday

# Drop original dates
df = df.drop(columns=["Order_Date", "Return_Date"])

In [30]:
df = df.drop(columns=["Return_Status", "Return_Date_present2"])

In [33]:
# Replace NaN already handled as "Not Applicable"
reason_counts = df["Return_Reason"].value_counts()
rare_reasons = reason_counts[reason_counts < 30].index  # adjust threshold

df["Return_Reason"] = df["Return_Reason"].replace(rare_reasons, "Other")

In [35]:
df.head(10)

,Product_Category,Product_Price,Order_Quantity,Return_Reason,User_Age,User_Gender,User_Location,Payment_Method,Shipping_Method,Discount_Applied,Return_Flag_fixed,Days_to_Return_filled2,Total_Order_Value,Order_Year,Order_Month,Order_Weekday
0,Clothing,1.114392,-0.004336,Changed mind,0.901627,Male,City54,Debit Card,Next-Day,1.411842,1,1.531264,0.789174,2023,8,5
1,Books,0.255540,-0.004336,Wrong item,1.554742,Female,City85,Credit Card,Express,1.587297,1,-0.532817,0.173866,2023,10,0
2,Toys,0.963493,1.417410,Not Applicable,-1.449591,Female,City30,Debit Card,Next-Day,0.114731,0,-0.712554,1.984889,2023,5,5
3,Toys,1.040902,-0.004336,Not Applicable,-0.273982,Male,City95,PayPal,Next-Day,-0.669942,0,-0.712554,0.736523,2024,8,3
4,Books,-0.995819,0.706537,Not Applicable,-0.665852,Female,City80,Gift Card,Standard,-0.600317,0,-0.712554,-0.538636,2023,1,0
5,Electronics,-0.001745,-1.426083,Defective,0.117887,Female,City47,Debit Card,Next-Day,1.574765,1,0.568799,-0.853268,2024,2,2
6,Clothing,0.913520,-0.715210,Wrong item,-1.253656,Male,City50,Credit Card,Express,0.243537,1,-0.329888,0.005285,2023,5,0
7,Electronics,0.378093,-0.004336,Not as described,1.489431,Female,City39,Credit Card,Express,0.969028,1,2.412557,0.261667,2023,2,3
8,Home,0.350167,1.417410,Defective,0.640380,Male,City81,Debit Card,Standard,-0.452016,1,-0.184938,1.252547,2024,12,2
9,Clothing,0.297954,-0.004336,Not as described,-0.012736,Male,City82,Credit Card,Standard,1.407665,1,1.125405,0.204253,2023,3,4


In [36]:
df["User_Location_Num"] = df["User_Location"].str.extract("(\d+)").astype(int)
df = df.drop(columns=["User_Location"])  # drop text column

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-193967500.py:1: SyntaxWarning: invalid escape sequence '\d'
  df["User_Location_Num"] = df["User_Location"].str.extract("(\d+)").astype(int)


In [37]:
df

,Product_Category,Product_Price,Order_Quantity,Return_Reason,User_Age,User_Gender,Payment_Method,Shipping_Method,Discount_Applied,Return_Flag_fixed,Days_to_Return_filled2,Total_Order_Value,Order_Year,Order_Month,Order_Weekday,User_Location_Num
0,Clothing,1.114392,-0.004336,Changed mind,0.901627,Male,Debit Card,Next-Day,1.411842,1,1.531264,0.789174,2023,8,5,54
1,Books,0.255540,-0.004336,Wrong item,1.554742,Female,Credit Card,Express,1.587297,1,-0.532817,0.173866,2023,10,0,85
2,Toys,0.963493,1.417410,Not Applicable,-1.449591,Female,Debit Card,Next-Day,0.114731,0,-0.712554,1.984889,2023,5,5,30
3,Toys,1.040902,-0.004336,Not Applicable,-0.273982,Male,PayPal,Next-Day,-0.669942,0,-0.712554,0.736523,2024,8,3,95
4,Books,-0.995819,0.706537,Not Applicable,-0.665852,Female,Gift Card,Standard,-0.600317,0,-0.712554,-0.538636,2023,1,0,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Home,-0.768980,0.706537,Not Applicable,-0.469917,Male,PayPal,Standard,0.645969,0,-0.712554,-0.321950,2023,10,4,40
9996,Electronics,1.625602,-0.004336,Not Applicable,1.620054,Male,Debit Card,Express,0.031181,0,-0.712554,1.155420,2023,2,5,62
9997,Toys,0.939276,1.417410,Not Applicable,0.117887,Male,Credit Card,Next-Day,-0.857929,0,-0.712554,1.955973,2024,5,4,74
9998,Toys,-0.861927,-1.426083,Not Applicable,-0.665852,Female,Gift Card,Express,1.739079,0,-0.712554,-1.058688,2024,2,1,34


In [38]:
cat_cols = ["Product_Category", "Return_Reason", "User_Gender",
            "Payment_Method", "Shipping_Method"]

for col in cat_cols:
    df[col] = df[col].astype("category").cat.codes

In [39]:
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Product_Category        10000 non-null  int8   
 1   Product_Price           10000 non-null  float64
 2   Order_Quantity          10000 non-null  float64
 3   Return_Reason           10000 non-null  int8   
 4   User_Age                10000 non-null  float64
 5   User_Gender             10000 non-null  int8   
 6   Payment_Method          10000 non-null  int8   
 7   Shipping_Method         10000 non-null  int8   
 8   Discount_Applied        10000 non-null  float64
 9   Return_Flag_fixed       10000 non-null  int64  
 10  Days_to_Return_filled2  10000 non-null  float64
 11  Total_Order_Value       10000 non-null  float64
 12  Order_Year              10000 non-null  int32  
 13  Order_Month             10000 non-null  int32  
 14  Order_Weekday           10000 non-null 

In [40]:
'''
Save the pre-processing dataset
'''
df_save_path = '/content/drive/MyDrive/datasets/ecommerce_returns_synthetic_data_preprocessed.csv'
df.to_csv(df_save_path, index=False)
